In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd

In [51]:
# import training data
df_data_train = pd.read_excel("drive/MyDrive/Progetto Data Mining 2/CODICE PROGETTO/Outliers/DATASET NO OUTLIERS/df_prep_TRAIN_no_outliers.xlsx", index_col="Unnamed: 0")
df_info_train = pd.read_excel("drive/MyDrive/Progetto Data Mining 2/CODICE PROGETTO/Outliers/DATASET NO OUTLIERS/df_prep_info_TRAIN_no_outliers.xlsx", index_col="Unnamed: 0")

# get training values
X_train = df_data_train.values
y_train = df_info_train['emotion']

# import test data
df_data_test = pd.read_excel("drive/MyDrive/Progetto Data Mining 2/CODICE PROGETTO/Outliers/DATASET NO OUTLIERS/df_prep_TEST_no_outliers.xlsx", index_col="Unnamed: 0")
df_info_test = pd.read_excel("drive/MyDrive/Progetto Data Mining 2/CODICE PROGETTO/Outliers/DATASET NO OUTLIERS/df_prep_info_TEST_no_outliers.xlsx", index_col="Unnamed: 0")

# get test values
X_test = df_data_test.values
y_test = df_info_test['emotion']

In [52]:
df_data_train

,sum,std,q25,kur,skew,lag1_sum,lag1_kur,lag1_skew,zc_sum,mfcc_sum,...,mfcc_q05_w4,mfcc_q95_w4,mfcc_q99_w4,mfcc_kur_w4,sc_std_w4,sc_kur_w4,sc_skew_w4,stft_sum_w4,stft_mean_w4,stft_kur_w4
0,0.716365,-1.159462,1.215433,0.859870,1.608196,0.032995,0.662262,0.805546,-1.500797,0.399234,...,0.914663,-0.670034,-1.541948,-0.202342,-0.425145,3.147197,-2.200824,0.228251,1.542380,-0.159600
1,0.669143,-1.108453,1.215433,1.070644,1.658463,0.032995,0.275624,1.241144,-1.321006,0.408776,...,0.465963,-0.925068,-0.828006,0.275867,-0.427448,-0.229617,-0.575310,-0.231871,1.094347,0.111549
2,0.723227,-0.975264,2.047744,1.146381,1.781550,-1.036084,0.763118,-0.128356,-1.750464,0.490061,...,0.637260,-0.595362,-1.139805,-0.014546,-0.246588,2.742928,-1.781613,-0.390360,1.161032,-0.289829
3,0.708504,-1.006749,1.215433,1.749201,1.981405,-0.815534,0.707610,1.132806,-1.416084,0.750825,...,1.016104,-0.098577,-0.717890,-0.392469,-0.555570,-0.462721,-0.367839,-0.824689,1.017073,0.412375
4,0.705644,-1.371989,1.215433,0.581488,1.436854,-0.815534,0.370417,-0.060381,-1.569077,-0.145204,...,0.736785,-0.583115,-0.797341,0.154515,-0.249430,0.605878,-0.549321,0.241436,1.091298,-0.159600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1773,-0.533551,-0.926692,2.047744,-0.775907,-0.974597,0.032995,0.234861,-0.828073,-1.373922,-0.346083,...,0.664345,-1.699436,-1.259249,0.894243,1.572822,1.191457,-0.871113,-0.858915,0.261546,-0.157863
1774,-0.512702,-0.494895,1.215433,-0.050756,-0.482154,0.032995,1.144182,-1.138132,-0.648233,0.011200,...,0.339675,-2.119495,-0.552764,1.389867,1.466784,1.206284,-0.742265,-1.242542,-0.273897,-0.282172
1775,-0.602680,-0.374843,2.047744,-0.945497,-0.285060,0.032995,0.858489,-0.565557,-1.728149,-0.008234,...,-0.089512,-1.531537,-0.442736,1.290250,0.875711,1.581088,-0.871506,-1.276408,-0.053277,0.231495
1776,-0.421299,-0.160226,1.215433,-1.368889,-0.548836,0.032995,-0.013708,-2.119959,0.007696,-0.385782,...,-0.377597,-1.926863,-1.238323,1.709477,1.727957,-0.044882,-0.627522,-0.587311,-0.180552,0.713603


#Adding Dropout

In [53]:
X_train.shape

(1778, 96)

In [93]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [94]:
# prepare target
def prepare_targets(y_train, y_test):
 le = OneHotEncoder(
    sparse_output=False
 )
 le.fit(y_train)
 y_train_enc = le.transform(y_train)
 y_test_enc = le.transform(y_test)
 return y_train_enc, y_test_enc

In [86]:
y_train = y_train.values.reshape((len(y_train), 1))
y_test = y_test.values.reshape((len(y_test), 1))

AttributeError: ignored

In [95]:
y_train_enc, y_test_enc = prepare_targets(y_train, y_test)
y_train_enc

array([[0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [117]:
# Network and training parameters.

# le epoche sono un'unità di misura per indicare quanto deve andare avanti l'addestramento
EPOCHS = 300
# il numero di samples che diamo in pasto alla rete neurale
BATCH_SIZE = 128
# numero di output (classi)
NB_CLASSES = 8
# numero di neuroni dello strato nascosto
N_HIDDEN = 64
# quanta parte del dataset viene riservata come validation set
VALIDATION_SPLIT = 0.1
# % of neurons to drop
DROPOUT = 0.3

VERBOSE = 1

In [118]:
# Build the model.
model = tf.keras.models.Sequential()

model.add(keras.layers.Dense(
    N_HIDDEN,
    input_shape=(96,),
    name='dense_layer',
    activation='relu'
    )
)

model.add(keras.layers.Dropout(
    DROPOUT
    )
)

model.add(keras.layers.Dense(
    NB_CLASSES,
    name='dense_layer_3',
    activation='softmax'
    )
)

# Compiling the model.
model.compile(
    optimizer='SGD',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Training the model.
model.fit(
    X_train, 
    y_train_enc,
    batch_size=BATCH_SIZE, 
    epochs=EPOCHS,
    verbose=VERBOSE, 
    validation_split=VALIDATION_SPLIT
)

Epoch 1/300
13/13 [==============================] - 0s 14ms/step - loss: 2.5615 - accuracy: 0.1412 - val_loss: 2.4236 - val_accuracy: 0.1573
Epoch 2/300
13/13 [==============================] - 0s 3ms/step - loss: 2.4231 - accuracy: 0.1531 - val_loss: 2.3193 - val_accuracy: 0.1629
Epoch 3/300
13/13 [==============================] - 0s 3ms/step - loss: 2.3398 - accuracy: 0.1681 - val_loss: 2.2451 - val_accuracy: 0.1854
Epoch 4/300
13/13 [==============================] - 0s 3ms/step - loss: 2.2760 - accuracy: 0.1838 - val_loss: 2.1853 - val_accuracy: 0.1966
Epoch 5/300
13/13 [==============================] - 0s 3ms/step - loss: 2.2081 - accuracy: 0.1906 - val_loss: 2.1384 - val_accuracy: 0.2135
Epoch 6/300
13/13 [==============================] - 0s 5ms/step - loss: 2.1738 - accuracy: 0.2013 - val_loss: 2.1000 - val_accuracy: 0.2191
Epoch 7/300
13/13 [==============================] - 0s 4ms/step - loss: 2.1198 - accuracy: 0.2075 - val_loss: 2.0675 - val_accuracy: 0.2191
Epoch 8/300


KeyboardInterrupt: ignored

In [82]:
#evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)

ValueError: ignored

In [ ]:
predictions = model.predict(X_test)